In [62]:
import pandas as pd
import xlrd
import arcpy,openpyxl
from pandas import ExcelWriter
from datetime import datetime
import os
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
from shutil import copyfile
from openpyxl.utils.cell import get_column_letter

In [63]:
def importallsheets(in_excel, out_gdb):
    ###this function import all sheets of an xlsx file into separate tables inside a GDB (with same name)
    print("max_counter: %s" % max_counter)
    counter = 0
    workbook = xlrd.open_workbook(in_excel)
    sheets = [sheet.name for sheet in workbook.sheets()]

    print('{} sheets found: {}'.format(len(sheets), ','.join(sheets)))
    for sheet in sheets:
        counter +=1
        if counter <= max_counter:
            # The out_table is based on the input excel file name
            # a underscore (_) separator followed by the sheet name
            out_table = os.path.join(
                out_gdb,
                arcpy.ValidateTableName(
                    "{0}".format(sheet),
                    out_gdb))

            print('Converting {} to {}'.format(sheet, out_table))

            # Perform the conversion
            arcpy.ExcelToTable_conversion(in_excel, out_table, sheet)
            

def make_attribute_dict(fc, code_field, value_field):
    #this function creates a dictionary based on a GDB table
    attdict = {}
    with arcpy.da.SearchCursor(fc, [code_field, value_field]) as cursor:
        for row in cursor:
            attdict[row[0]] = row[1]
    return attdict

def fix_category_formatting(category):
    #this function improves and standardizes the formatting of the categories' descriptions
     return category.replace("[","(").replace("]",")").replace("(specify)","").replace("/ ",", ").capitalize().replace("adps","ADPs").replace("idp","IDP").replace("covid","COVID").replace(" , ",", ").replace("staplec","staple") 

def count_number_of_questions_qname(questionnaire_file):
    questionnaire_df = pd.read_excel(pd.ExcelFile(questionnaire_file), sheet_name='survey',skiprows=2)
    questionnaire_df = questionnaire_df[['Q Name']]
    list_questions = questionnaire_df.dropna().values.tolist()
    list_questions = [item for sublist in list_questions for item in sublist]
    n_of_questions = len(list_questions)
    return list_questions, n_of_questions

def count_number_of_questions_sqname(questionnaire_file):
    questionnaire_df = pd.read_excel(pd.ExcelFile(questionnaire_file), sheet_name='survey',skiprows=2)
    questionnaire_df = questionnaire_df[['Suggested Qname']]
    list_questions = questionnaire_df.dropna().values.tolist()
    list_questions = [item for sublist in list_questions for item in sublist]
    n_of_questions = len(list_questions)
    return list_questions, n_of_questions
    

def read_questionnaire(input_questionnaire_file, writer):
    ##this section of the script reads the survey excel file and creates an excel file with multiple sheets:
    ### each sheet contains the coded value and description for a "Single choice" or "Open Ended-Select All That Apply" question.
    ###moreover, it creates and populates several lists that will be used later for defining each field of the final table names, types and domains

    #print("Opening questionnaire DF")
    quest_df = pd.read_excel(open(input_questionnaire_file, 'rb'), sheet_name='survey',skiprows=2)
    #create a list of all possible numbering
    numbering = ["%s)" % n for n in range(1,200)] ## 1), 2), ... 200)
    # initialize list of lists that will store the results
    dict_derived_fieldnames = {} #this dict will group all derived fields in case of "Select All That Apply" type of questions
    field_names_list = [] ##this list will contain all fields of the final table
    text_type_fields = [] ##this list will contain all fields of the final table with TEXT type
    range_type_fields = [] ##this list will contain all fields of the final table storing RANGE data  (will be LONG type)
    double_type_fields = [] ##this list will contain all fields of the final table storing DOUBLE data  
    ##iterate the following for each row (so each question of the questionnaire)
    all_derived_fieldnames = []
    all_answers_with_other_option = [] ##this list will contain all "Other: specify" fields
    quest_df = quest_df[quest_df['Suggested Qname'].notna()]
    for index, row in quest_df.iterrows():
        question_name = []
        question_type = []
        try:
            first_derived_fieldname = "" #the name of the first derived field will be the main of the domain table
            derived_fieldnames = []
            codes_and_labels = []
            categories = str(row['English']).replace("\t","")
            question_name = row['Suggested Qname'].strip()  #Q Name
            question_type = row['Q Type']
            programming_instructions = row['Programming Instructions'] #this field contains coded values for crop_main
            #print("\n\n----%s----" % question_name)
            #only for questions with pre-defined categories need domains
            if question_type in ("StartRecording","Single Choice","Open Ended-Single Choice", "Open Ended - Single Choice", "Open Ended-Select All That Apply",
                                 "Select All That Apply","Open Ended - Select All That Apply "):
                if question_name == 'crp_main': #for this question only, coded values should be taken from field programming_instructions
                    programming_lines = programming_instructions.splitlines()
                    for programming_line in programming_lines:
                        if ")" in programming_line:
                            #print(programming_line)
                            index, category = programming_line.split(")")
                            category = fix_category_formatting(category)
                            codes_and_labels.append([index, category])
                else:
                    #find all numbering present in the category string
                    numbering_in_text = [n for n in numbering if n in categories]
                    #print(numbering_in_text)
                    ##the following loop creates a list "codes_and_labels" with all available codes&labels for each question
                    for index in range(0,len(numbering_in_text)):
                        start = categories.find(numbering_in_text[index]) + len(numbering_in_text[index])
                        try:
                            end = categories.find(numbering_in_text[index + 1])
                            substring = categories[start:end].strip()
                        except:
                            # it fails during the last loop -> the last option is usually at the end of the string
                            substring = categories[start:].strip()
                        #print(substring)
                        category = fix_category_formatting(substring)
                        codes_and_labels.append([index +1, category])

                if question_type not in ["Open Ended-Select All That Apply","Select All That Apply","Open Ended - Select All That Apply "]:
                    #so questions with NO derived fields
                    field_names_list.append(question_name.strip())
                    codes_and_labels_df = pd.DataFrame(codes_and_labels, columns=['code', 'label'])
                    codes_and_labels_df.to_excel(writer, sheet_name=question_name)
                else:
                    #so questions with derived fields
                    numbering_in_qname = [n for n in numbering if n in question_name]
                    for index in range(0, len(numbering_in_qname)):
                        start = question_name.find(numbering_in_qname[index]) + len(numbering_in_qname[index])
                        try:
                            end = question_name.find(numbering_in_qname[index + 1])
                            derived_field_name = question_name[start:end].strip()
                        except:
                            # it fails during the last loop -> the last option is usually at the end of the string
                            derived_field_name = question_name[start:].strip()
                        all_derived_fieldnames.append(derived_field_name)
                        derived_fieldnames.append(derived_field_name)
                        field_names_list.append(derived_field_name)
                        if derived_field_name[-6:] == "_other": ##this field will need to be STRING - with no domain (since it's a 'other specify')
                            all_answers_with_other_option.append(derived_field_name)
                        if index == 0:
                            first_derived_fieldname = derived_field_name
                            codes_and_labels_df = pd.DataFrame(codes_and_labels, columns=['code', 'label'])
                            #codes_and_labels_df.to_excel(writer, sheet_name=derived_field_name) #we don't need domain table for derived fields, since they will use YES NOT domain table
                    dict_derived_fieldnames[first_derived_fieldname] = all_derived_fieldnames
            elif question_type == "Range":
                #these questions will be associated to LONG type fields
                field_names_list.append(question_name.strip())
                range_type_fields.append(question_name)
            elif question_type == "Open Ended":
                #these questions will be associated to TEXT type fields
                if not pd.isnull(question_name): #NaN rows we want to skip (i.e. OptIn question without a name in the survey)
                    field_names_list.append(question_name)
                    text_type_fields.append(question_name)
            else:
                #print("QUESTION SKIPPED ---------", question_name, question_type)
                pass
        except Exception as e: 
            print("Failed question %s (type: %s) for error: %s " % (question_name, question_type,e))

    #adding Yes No table (for derived fields domain)
    d = {1:"Yes",0:"No"}
    yesno_df = pd.DataFrame(d.items(), columns=['code', 'label'])
    yesno_df.to_excel(writer, sheet_name='yes_no')
    #print("Saving codes and labels %s" % coded_values_file)

    #print(all_answers_with_other_option)

    #creating additional sheet with all derived fields, that will be used for a script that ensures that also these values are within the domains in the output table, in a later stage
    list_of_yes_no_fields = []
    for derived_field in all_derived_fieldnames:
        if derived_field not in all_answers_with_other_option:
            list_of_yes_no_fields.append(derived_field)

    derived_fields_df = pd.DataFrame(list_of_yes_no_fields) 
    derived_fields_df.to_excel(writer, sheet_name='derived_fields') 
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    
def detect_enumerator(geopoll_or_kobo_template):
    enumerator = ""
    if "kobo" in geopoll_or_kobo_template.lower():
        country_df.columns = country_df.columns.str.replace("[/]", "")
        enumerator = "kobo"
    elif "geopoll" in geopoll_or_kobo_template.lower():
        enumerator = "geopoll"
    else:
        print("Please include Geopoll or Kobo inside the filename: %s " % geopoll_or_kobo_template)
        enumerator = "Please include Geopoll or Kobo inside the filename"
    return enumerator

def insert_sheet_with_adm2_reference(questionnaire_file,adm0_iso3):
    gis = GIS("https://hqfao.maps.arcgis.com")
    item = gis.content.get('3596c3ad318849068eda21517ade30be')
    flayer = item.layers[0]
    #queryies content of interest
    query = "adm0_ISO3 = '" + adm0_iso3 +"'"
    sdf = flayer.query(where=query).sdf
    #print(sdf.head())
    del sdf['OBJECTID']
    del sdf['validity']
    del sdf['Shape__Area']
    del sdf['Shape__Length']
    del sdf['SHAPE']
    #print(sdf.head())
    excel_book = openpyxl.load_workbook(questionnaire_file)
    with pd.ExcelWriter(questionnaire_file, engine='openpyxl') as writer:
        writer.book = excel_book
        sdf.to_excel(writer, 'ADMIN info', index=False)
        writer.save()
        
    return sdf

def find_and_replace_strings_in_df(questionnaire_file):
    replacing_table_df = pd.read_excel(pd.ExcelFile(questionnaire_file), 'Additional information', skiprows=1)
    replacing_table_df = replacing_table_df[['Original','Replacement']]

    workbook = openpyxl.load_workbook(questionnaire_file)
    workbook.sheetnames
    worksheet = workbook["survey"]

    number_of_rows = worksheet.max_row #Number of Rows
    number_of_columns = worksheet.max_column #Number of Columns

    replacementTextKeyPairs = {'$ADMIN1$': replacing_table_df.iloc[0]['Replacement'],
                               '$ADMIN2$': replacing_table_df.iloc[1]['Replacement'],
                               '$currency$': replacing_table_df.iloc[2]['Replacement'],
                               '$MIN SALARY$': replacing_table_df.iloc[3]['Replacement'],
                               '$MAX SALARY$': replacing_table_df.iloc[4]['Replacement'],
                               '$THRESHOLD$': replacing_table_df.iloc[5]['Replacement'],
                               '$season$': replacing_table_df.iloc[6]['Replacement'],
                               '$expected or nothing$': replacing_table_df.iloc[7]['Replacement']}
    # Iterate over the columns and rows, search
    # for the text and replace
    for i in range(number_of_columns):
        for k in range(number_of_rows):
          
            cellValue = str(worksheet[get_column_letter(i+1)+str(k+1)].value)
            #print(cellValue)
            
            for key in replacementTextKeyPairs.keys():
              
                if key in str(cellValue) and str(cellValue) != None:
                    newCellValue = cellValue.replace(key,replacementTextKeyPairs.get(key))
                    #newCellValue = replacementTextKeyPairs.get(key)
                    worksheet[get_column_letter(i+1)+str(k+1)] = str(newCellValue) 
    workbook.save(questionnaire_file)
    
def sort_crop_list_by_selection(questionnaire_file):
    crop_list_df = pd.read_excel(pd.ExcelFile(questionnaire_file), 'Crop list', skiprows=2)
    crop_list_df = crop_list_df[['Select top 10 crops ','GeoPoll code','Dataset code','Label']]
    q_name = "crp_main"
    q_label = "What has been the main crop that your household has grown for food and income generation in $season$, if any? \n [OPERATOR: SINGLE RESPONSE. \"NO CROP PRODUCTION\" IS IN LAST OPTIONS AT THE END.] \n \n"
    
    workbook = openpyxl.load_workbook(questionnaire_file)
    workbook.sheetnames
    worksheet = workbook["survey"]
    
    number_of_rows = worksheet.max_row #Number of Rows
    number_of_columns = worksheet.max_column #Number of Columns
    


    list_choices = crop_list_df['Select top 10 crops '].dropna().values.tolist()
    n_of_choices = len(list_choices)


    
    sorted_crop_list_df = crop_list_df.sort_values(by = ['Select top 10 crops ','GeoPoll code'])
    #print(sorted_crop_list_df)
    
    sorted_crop_list_df['combined'] = sorted_crop_list_df['GeoPoll code'].astype(str)+')'+sorted_crop_list_df['Label']
    sorted_crop_list = sorted_crop_list_df[['combined']].values.tolist()
    #print(sorted_crop_list)
    final_sorted_crop_list = '\n'.join(map(str, [i[0] for i in sorted_crop_list]))

    #print(final_sorted_crop_list)
    #print(crop_list_df)

    # Iterate over the columns and rows, search
    # for the text and replace

    for k in range(number_of_rows):
          
        questionValue = str(worksheet[get_column_letter(2)+str(k+1)].value)
        labelValue = str(worksheet[get_column_letter(4)+str(k+1)].value)
            #print(cellValue)
              
        if q_name in str(questionValue) and str(questionValue) != None:
              newLabelValue = q_label + final_sorted_crop_list
              #newCellValue = replacementTextKeyPairs.get(key)
              worksheet[get_column_letter(4)+str(k+1)] = str(newLabelValue) 
    workbook.save(questionnaire_file)
    return n_of_choices


    

In [64]:

###USER DEFINED INPUTS BEGINNING
iso3 = "PHL"
country_questionnaire_file = r'C:\temp\household_questionnaire_geopoll_20210525_PH.xlsx' #the questionnaire file that we use for creating the table
template_questionnaire_file = r'C:\git\hh_survey\household_questionnaire_geopoll_202105.xlsx' #the questionnaire file that we use for creating the table
output_locations = r'C:\temp\data_processing_exports'
###USER DEFINED INPUTS END

execution_messages = ''

enumerator = detect_enumerator(country_questionnaire_file)

print("Reading TEMPLATE questionnaire and creating recap excel file")
questionnaire_file = template_questionnaire_file
temp_path = output_locations

startTime = datetime.now()
now = datetime.now().strftime('%Y%m%d%H%M%S')
coded_values_file = os.path.join(temp_path, "coded_values_%s_template.xlsx" % now) #intermediary output file with all categories and codes extracted from the questionnaire
writer = pd.ExcelWriter(coded_values_file, engine='xlsxwriter')
field_names_list = []
max_counter = 3000 #for testing purposes, we may need to limit the execution only to some items
read_questionnaire(questionnaire_file, writer)

print("Reading COUNTRY questionnaire and creating recap excel file")
country_coded_values_file = os.path.join(temp_path, "coded_values_%s_country.xlsx" % now) #intermediary output file with all categories and codes extracted from the questionnaire
country_writer = pd.ExcelWriter(country_coded_values_file, engine='xlsxwriter')
field_names_list = []
max_counter = 3000 #for testing purposes, we may need to limit the execution only to some items
read_questionnaire(country_questionnaire_file, country_writer)

Reading TEMPLATE questionnaire and creating recap excel file
Reading COUNTRY questionnaire and creating recap excel file


In [65]:
print ("COUNTING NUMBER OF QUESTIONS")

T_sqname_list_questions, T_sqname_n_of_questions = count_number_of_questions_sqname(questionnaire_file)
T_qname_list_questions, T_qname_n_of_questions = count_number_of_questions_qname(questionnaire_file)
C_sqname_list_questions, C_sqname_n_of_questions = count_number_of_questions_sqname(country_questionnaire_file)
C_qname_list_questions, C_qname_n_of_questions = count_number_of_questions_qname(country_questionnaire_file)

message1 = "QName Field - Number of questions in the Template Questionnaire: %s" % T_qname_n_of_questions
message2 = "QName Field - Number of questions in the Country Questionnaire: %s\n" % C_qname_n_of_questions
message3 = "SuggestedQName Field - Number of questions in the Template Questionnaire: %s" % T_sqname_n_of_questions
message4 = "SuggestedQName Field - Number of questions in the Country Questionnaire: %s" % C_sqname_n_of_questions
execution_messages += "\n%s\n%s\n%s\n%s" % (message1, message2, message3, message4)
print(message1)
print(message2)
print(message3)
print(message4)

COUNTING NUMBER OF QUESTIONS
QName Field - Number of questions in the Template Questionnaire: 126
QName Field - Number of questions in the Country Questionnaire: 129

SuggestedQName Field - Number of questions in the Template Questionnaire: 124
SuggestedQName Field - Number of questions in the Country Questionnaire: 124


In [66]:
print ("DETECTING DIFFERENCES IN FIELD: Suggested QName")

message5 = "\nQuestions in country questionnaire and not in template questionnaire: %s" % list(set(C_sqname_list_questions) - set(T_sqname_list_questions))
message6 = "Questions in template questionnaire and not in country questionnaire: %s" % list(set(T_sqname_list_questions) - set(C_sqname_list_questions))
execution_messages += "\n%s\n%s" % (message5, message6)
print (message5)
print (message6)


DETECTING DIFFERENCES IN FIELD: Suggested QName

Questions in country questionnaire and not in template questionnaire: ['hh_wealth_water']
Questions in template questionnaire and not in country questionnaire: ['hh_wealth_water\n\n1)hh_wealth_water_pvttap\n2)hh_wealth_water_publictap\n3)hh_wealth_water_protectwell\n4)hh_wealth_water_bottle\n5)hh_wealth_water_othersafe\n6)hh_wealth_water_river\n7)hh_wealth_water_unprotectwell\n8)hh_wealth_water_spring\n9)hh_wealth_water_canalsurface\n10)hh_wealth_water_otherunsafe\n11)hh_wealth_water_dk\n12)hh_wealth_water_ref']


In [67]:
print ("DETECTING DIFFERENCES IN FIELD: Q Name")

message7 = "\nQuestions in country questionnaire and not in template questionnaire: %s" % list(set(C_qname_list_questions) - set(T_qname_list_questions))
message8 = "Questions in template questionnaire and not in country questionnaire: %s" % list(set(T_qname_list_questions) - set(C_qname_list_questions))
execution_messages += "\n%s\n%s" % (message7, message8)
print (message7)
print (message8)

DETECTING DIFFERENCES IN FIELD: Q Name

Questions in country questionnaire and not in template questionnaire: ['hh_head', 'crp_landdoc']
Questions in template questionnaire and not in country questionnaire: []


In [68]:
print ("CHECKING THAT ALL MANDATORY QUESTIONS ARE INCLUDED IN THE COUNTRY QUESTIONNAIRE")

quest_df = pd.read_excel(open(questionnaire_file, 'rb'), sheet_name='survey',skiprows=2)
#print(list(quest_df.columns.values))
quest_df.rename(columns={'Unnamed: 19': 'Mandatory'}, inplace=True)
list_mandatory_questions = quest_df.loc[quest_df["Mandatory"] == "yes"]["Q Name"].values.tolist() 
message9 = "\nFound %s mandatory questions in the template questionnaire" % len(list_mandatory_questions)
print(message9)

result =  all(elem in C_qname_list_questions  for elem in list_mandatory_questions)
if result:
    message10 = "Yes, country questionnaire contains all mandatory questions"   
else:
    message10 = "No, country questionnaire does not contains all mandatory questions. Missing question: %s " % list(set(list_mandatory_questions ) - set(C_qname_list_questions))
print(message10)
    
execution_messages += "\n%s\n%s" % (message9, message10)

CHECKING THAT ALL MANDATORY QUESTIONS ARE INCLUDED IN THE COUNTRY QUESTIONNAIRE

Found 63 mandatory questions in the template questionnaire
Yes, country questionnaire contains all mandatory questions


In [69]:
print ("CHECKING THAT RULES ON CS QUESTIONS ARE RESPECTED\n")

cs_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs')]
if len(cs_questions_in_country) > 0:
    cs_stress_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs_stress')]
    cs_emergency_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs_emergency')]
    cs_crisis_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs_crisis')]
    message11 = "\n%s CS questions found in the country questionnaire\n" % len(cs_questions_in_country) + "There should be 3 questions for each group.\nNumber of STRESS questions: %s" % len(cs_stress_questions_in_country) + "\nNumber of CRISIS questions: %s\n" % len(cs_crisis_questions_in_country) + "Number of EMERGENCY questions: %s" % len(cs_emergency_questions_in_country)

else:
    message11 = "\nNo CS questions in the country questionnaire"

print(message11)

execution_messages += "\n%s" % (message11)

CHECKING THAT RULES ON CS QUESTIONS ARE RESPECTED


17 CS questions found in the country questionnaire
There should be 3 questions for each group.
Number of STRESS questions: 6
Number of CRISIS questions: 6
Number of EMERGENCY questions: 5


In [70]:
print ("CHECKING THAT RULES ON HDDS QUESTIONS ARE RESPECTED")

hdds_questions_in_country = [i for i in C_qname_list_questions if i.startswith('hdds')]
message12 = "\nThere should be either 0 or 2 HDDS questions"

if len(hdds_questions_in_country) > 0:
    message12 += "\nNumber of HDDS questions: %s  - %s" % (len(hdds_questions_in_country), hdds_questions_in_country)
else:
    message12 += "No HDDS questions in the country questionnaire"
    execution_messages += "\n%s" % (message12)

print(message12)

CHECKING THAT RULES ON HDDS QUESTIONS ARE RESPECTED

There should be either 0 or 2 HDDS questions
Number of HDDS questions: 2  - ['hdds', 'hdds_confirmation']


In [71]:
print("COMPARING DOMAINS AND DERIVED FIELDS")
all_domains_respected = True
domain_messages = ''
dict_with_dfs_template = pd.read_excel(coded_values_file, sheet_name=None)
dict_with_dfs_country = pd.read_excel(country_coded_values_file, sheet_name=None)
for template_question, template_domains in dict_with_dfs_template.items():
    if template_question in dict_with_dfs_country:
        #print(template_question)
        if template_question == 'derived_fields':
            domain_messages += "\n\n-------\n"
            domain_messages += "\nDerived fields analysis: "
            derived_fields_country = list(dict_with_dfs_country[template_question][0].tolist())
            derived_fields_template = list(template_domains[0].tolist())
            derived_fields_only_country = list(set(derived_fields_country ) - set(derived_fields_template))
            derived_fields_only_template = list(set(derived_fields_template ) - set(derived_fields_country))
            if len(derived_fields_only_country) == 0 and len(derived_fields_only_template) == 0:
                domain_messages += "\nDerived fields have been respected"
            else:
                domain_messages += "\nDerived field only in template: %s" % derived_fields_only_template
                domain_messages += "\nDerived field only in country: %s" % derived_fields_only_country
        elif template_question == 'crp_main':
            pass ##crp domains are going to be edited by the script - no need to check the now#
        else:
            df_diff = pd.concat([template_domains,dict_with_dfs_country[template_question]]).drop_duplicates(keep=False)
            del df_diff['Unnamed: 0']
            if not df_diff.empty:
                domain_messages += "\n\n-------\n"
                all_domains_respected = False
                domain_messages += "\nDomains have been changed for question: %s" % template_question
                #print("First row is the template domain, second row is the country domain")
                with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
                    #print(df_diff)
                    domain_messages += "\n%s" % df_diff.to_string()
            
            
if all_domains_respected == True:
    domain_messages += "\nAll domains have been respected"

print (domain_messages)
execution_messages += "\n%s" % (domain_messages)

COMPARING DOMAINS AND DERIVED FIELDS


-------

Domains have been changed for question: hh_education
   code                                 label
4     5  Islamic or other religious education
4     5                     Islamic education

-------

Domains have been changed for question: hh_wealth_toilet
   code                              label
0     1  Flush latrine (toilet with water)
5     6                         Don't know
6     7                            Refused
0     1    Flush latrine/toilet with water

-------

Domains have been changed for question: hh_wealth_light
   code       label
5     6  Don'tk now
6     7     Refused

-------

Domains have been changed for question: income_sec
   code                                        label
0     1  Farmer, production and sale of cereal crops
0     1  Farmer, production and sale of staple crops

-------

Domains have been changed for question: income_third
    code                                        label
0      1  Farmer

In [72]:
print ("CREATE OUTPUT VALIDATED QUESTIONNAIRE FILE")
iso3 = "MLI" #####to be removed
country_questionnaire_edited_file = os.path.join(temp_path, "validated_questionnaire_geopoll_%s_%s.xlsx" % (iso3, now)) 
copyfile(country_questionnaire_file, country_questionnaire_edited_file)
execution_messages += "\n\nOutput validated qcountry questionnaire: %s" % (country_questionnaire_edited_file)

CREATE OUTPUT VALIDATED QUESTIONNAIRE FILE


In [73]:
print("INSERT ADMIN SHEET WITH UPDATED BOUNDARIES")

sdf = insert_sheet_with_adm2_reference(country_questionnaire_edited_file,iso3)
sdf.head()
execution_messages += "\n\nCreated Admin Info sheet with updated boundaries: %s" % (country_questionnaire_edited_file)

INSERT ADMIN SHEET WITH UPDATED BOUNDARIES


In [74]:
print("EDITING THE SURVEY BY SORTING THE CROP LIST VALUES ACCORDING TO PRIORITISE")
n_of_choices = sort_crop_list_by_selection(country_questionnaire_edited_file)
message13 = "\nNumber of most common crops selected in the Crop list sheet: %s" % n_of_choices
print(message13)
execution_messages += "\n\nSurvey edited by sorting the crop list according to the prioritise" + message13

EDITING THE SURVEY BY SORTING THE CROP LIST VALUES ACCORDING TO PRIORITISE

Number of most common crops selected in the Crop list sheet: 10


In [75]:
print("EDITING THE SURVEY BY REPLACING VALUES ACCORDING ADDITIONAL INFO SHEET")
find_and_replace_strings_in_df(country_questionnaire_edited_file)
execution_messages += "\n\nSurvey edited by replacing values according the Additional Information sheet"

EDITING THE SURVEY BY REPLACING VALUES ACCORDING ADDITIONAL INFO SHEET


In [76]:
print("Create output file with execution messages")
text_file = os.path.join(temp_path, "questionnaire_validation_report_%s_%s.txt" % (iso3, now)) 
file = open(text_file, "w") 
file.write(execution_messages) 
file.close() 

Create output file with execution messages


In [77]:
print("Execution time: ", datetime.now() - startTime)

Execution time:  0:00:07.812931
